In [2]:
### IMPORTS ###

In [2]:
# Libraries
import pandas as pd
import numpy as np
import load_datasets
import re
import ast
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime, timezone

In [3]:
# County Data - Load county shapefile (California only)
counties = gpd.read_file("resources/cb_2018_us_county_500k.shp")
counties_ca = counties[counties["STATEFP"] == "06"]  # 06 = California
counties_ca = counties_ca.sort_values("NAME").reset_index(drop=True)
counties_ca["COUNTY_NUM"] = counties_ca.index

In [ ]:
# Import data into notebook
cafes, users, reviews = load_datasets.load_table_data()
ratings, user2cafes, cafes2users = load_datasets.create_user_review_dicts(reviews)

In [ ]:
### SECTION 1 - DATA ANALYTICS ###
# This section looks at various statistics about the data

In [ ]:
# Basics:

# Get number of users
print("Number of Users: ", len(users))
# Get number of cafes
print("Number of Cafes: ", len(cafes))
# Get number of reviews
print("Number of Reviews: ", len(reviews))

# Format of User Data
print("\nUSER EXAMPLE DATA: \n", users.iloc[0,:], "\n")
# Format of Cafe Data
print("CAFE EXAMPLE DATA: \n", cafes.iloc[0,:], "\n")
# Format of Review
print("REVIEW EXAMPLE DATA: \n", reviews.iloc[0,:], "\n")

Number of Users:  196454
Number of Cafes:  15576
Number of Reviews:  1769673

USER EXAMPLE DATA: 
 user_id        100000041656879737279
num_reviews                        7
Name: 0, dtype: object 

CAFE EXAMPLE DATA: 
 gmap_id                       0x80dc976f028eb61d:0x1a5ed32889a67122
name                                                       Circle K
latitude                                                  33.689862
longitude                                               -117.376151
category          ['Convenience store', 'ATM', 'Coffee shop', 'C...
avg_rating                                                      3.5
num_of_reviews                                                   24
price                                                             $
hours             [['Thursday', 'Open 24 hours'], ['Friday', 'Op...
Name: 0, dtype: object 

REVIEW EXAMPLE DATA: 
 gmap_id                  0x80dc976f028eb61d:0x1a5ed32889a67122
user_id                                  10898487406889310

In [6]:
# Functions:

# Average rating
def avg_rating(df):
    return sum(df['rating']) / len(df['rating'])

# Average price
def avg_price(df):
    return sum(df['price']) / len(df['price'])

# Converts time text to values
def parse_time(t):
    t = t.strip().upper()

    # Match hh or hh:mm formats
    m = re.match(r"(\d{1,2})(?::(\d{2}))?(AM|PM)", t)
    if not m:
        raise ValueError(f"Invalid time format: {t}")

    hour = int(m.group(1))
    minute = int(m.group(2) or 0)
    period = m.group(3)

    # Convert to 24-hour
    if period == "AM":
        if hour == 12:
            hour = 0
    else:  # PM
        if hour != 12:
            hour += 12

    return hour + minute / 60.0

# County Mapping Based on Latitude & Longitude
def get_county(lat, lon):
    point = Point(lon, lat)  # geometry expects (lon, lat)
    matches = counties_ca[counties_ca.contains(point)]

    if len(matches) > 0:
        return int(matches.iloc[0]["COUNTY_NUM"])
    return None

# Gets County Name Based on Latitude & Longitude
def get_county_name(lat, lon):
    county_num = get_county(lat,lon)
    return counties_ca[counties_ca["COUNTY_NUM"] == county_num]["NAME"]


In [ ]:
# Data Visualization:

# Formatting from price using '$' symbol

def price_to_num(p):
    if pd.isna(p):
        return np.nan
    p = str(p).strip()
    if p == "" or p.lower() == "none":
        return np.nan
    n = p.count("$")
    return n if n > 0 else np.nan

cafes["price_num"] = cafes["price"].apply(price_to_num)
cafes["price_num"].value_counts(dropna=False)


# Parsing hours if the cafe has them

def parse_time_token(tok):
    tok = tok.strip().upper() # cleaning text to parse
    
    # Checking 24h like 18:00
    m24 = re.match(r"^(\d{1,2}):(\d{2})$", tok) # trying to match 24-hour format
    if m24:
        hour, minute = int(m24.group(1)), int(m24.group(2))
        return hour*60 + minute

    # Checking AM/PM like 7AM, 7:30 PM
    m = re.match(r"^(\d{1,2})(?::(\d{2}))?\s*(AM|PM)$", tok)
    if not m:
        return None
    hour = int(m.group(1))
    minute = int(m.group(2)) if m.group(2) else 0
    ampm = m.group(3)
    if ampm == "PM" and hour != 12: hour += 12
    if ampm == "AM" and hour == 12: hour = 0
    return hour*60 + minute

# Converting intervals of hours to a consistent number
def interval_to_hours(interval_str):
    s = str(interval_str).replace("–", "-").replace("—", "-").strip()
    if s.lower() in ["closed", "none", "nan", ""]:
        return 0.0
    if "24 hours" in s.lower():
        return 24.0

    parts = [p.strip() for p in s.split("-")]
    if len(parts) != 2:
        return np.nan

    start = parse_time_token(parts[0])
    end = parse_time_token(parts[1])
    if start is None or end is None:
        return np.nan

    if end < start:  # Overnight
        end += 24*60
    return (end - start) / 60.0

# Summing weeks hours
def hours_to_weekly_total(hours_field):
    if pd.isna(hours_field): # returning nan if no hours exist for cafe
        return np.nan 
    try:
        data = ast.literal_eval(hours_field) if isinstance(hours_field, str) else hours_field
    except Exception:
        return np.nan
    if not isinstance(data, (list, tuple)):
        return np.nan

    total = 0.0
    for item in data:
        if not item or len(item) < 2:
            continue
        intervals = item[1]
        if isinstance(intervals, str):
            total += interval_to_hours(intervals)
        elif isinstance(intervals, (list, tuple)): # if the cafe has multiple intervals of open times
            for inter in intervals:
                total += interval_to_hours(inter)
    return total

cafes["weekly_hours"] = cafes["hours"].apply(hours_to_weekly_total)
cafes["avg_daily_hours"] = cafes["weekly_hours"] / 7.0

cafes[["hours","weekly_hours","avg_daily_hours"]].head(3)


# Combining cafes and reviews

df = reviews.merge(
    cafes[["gmap_id","name","latitude","longitude","price_num","avg_daily_hours","avg_rating"]],
    on="gmap_id",
    how="left"
)
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
df.head()



# Hours vs Rating - data prep

cafe_avg = df.groupby("gmap_id").agg(
    avg_user_rating=("rating","mean"),
    avg_daily_hours=("avg_daily_hours","first"),
).dropna()

bins_hours = pd.cut(cafe_avg["avg_daily_hours"], bins=10)
binned_hours = cafe_avg.groupby(bins_hours)["avg_user_rating"].mean()


# Review time vs Average Rating - data prep

reviews_time = reviews.copy()

# making sure the rating is numeric
reviews_time["rating"] = pd.to_numeric(reviews_time["rating"], errors="coerce")

# converting Unix ms to datetime
reviews_time["timestamp"] = pd.to_datetime(
    reviews_time["time"],
    unit="ms"
)

reviews_time["date"] = reviews_time["timestamp"].dt.date
reviews_time["month"] = reviews_time["timestamp"].dt.to_period("M").dt.to_timestamp()

# computing average rating per month and number of reviews per month
time_stats_all = (
    reviews_time
        .dropna(subset=["rating"])
        .groupby("month")
        .agg(
            avg_rating=("rating", "mean"),
            num_reviews=("rating", "count")
        )
        .reset_index()
)


min_reviews = 100  # change to change the number of minimum reviews needed
ts_global = time_stats_all[
    (time_stats_all["num_reviews"] >= min_reviews)
    & (time_stats_all["month"] >= "2008-01-01") # change to change time period, set at 2008 and after currently
].sort_values("month")


# Review Time vs Average Rating (3 Time Periods) - data prep

# Creating time boundaries // can change if we want
boundary_2016 = pd.Timestamp("2016-01-01")
boundary_2020 = pd.Timestamp("2020-01-01")

def unix_ms_to_period(unix_ms):
    """
      0 -> before 2016
      1 -> 2016-2019
      2 -> 2020 and later
    """
    if pd.isna(unix_ms):
        return np.nan
    try:
        t = int(unix_ms)
    except (ValueError, TypeError):
        return np.nan
    
    b2016_ms = int(pd.Timestamp("2016-01-01").timestamp() * 1000)
    b2020_ms = int(pd.Timestamp("2020-01-01").timestamp() * 1000)

    if t < b2016_ms:
        return 0
    elif t < b2020_ms:
        return 1
    else:
        return 2

# adding numeric time-period feature to reviews
reviews["time_period"] = reviews["time"].apply(unix_ms_to_period)

# Label function for plotting
def label_period(ts):
    if ts < boundary_2016:
        return "pre-2016"
    elif ts < boundary_2020:
        return "2016-2019"
    else:
        return "2020+"

time_stats_period = time_stats_all.copy()
time_stats_period["period"] = time_stats_period["month"].apply(label_period)


# Price vs Rating - data prep

cafe_avg_price = df.groupby("gmap_id").agg(
    avg_user_rating=("rating","mean"),
    price_num=("price_num","first")
).dropna()

levels = sorted(cafe_avg_price["price_num"].unique())
means = cafe_avg_price.groupby("price_num")["avg_user_rating"].mean()


# Creating big figure with all graphs

fig, axes = plt.subplots(3, 3, figsize=(18, 12))
axes = axes.ravel()

# 0: Open hours vs rating
ax = axes[0]
ax.scatter(cafe_avg["avg_daily_hours"], cafe_avg["avg_user_rating"], alpha=0.35)
ax.set_xlabel("Avg daily open hours")
ax.set_ylabel("Average review rating")
ax.set_title("Open hours vs rating (cafe-level)")

# 1: Binned open hours vs rating
ax = axes[1]
ax.plot(binned_hours.index.astype(str), binned_hours.values, marker="o")
ax.set_xlabel("Avg daily hours (binned)")
ax.set_ylabel("Mean rating")
ax.set_title("Binned open hours vs rating")
ax.tick_params(axis="x", rotation=45)

# 2: Average review rating over time
ax = axes[2]
ax.plot(ts_global["month"], ts_global["avg_rating"], marker="o", linestyle="-")
ax.set_xlabel("Review month")
ax.set_ylabel("Average rating")
ax.set_title("Average review rating over time")
ax.tick_params(axis="x", rotation=45)
ax.set_ylim(3.0, 5.0)  # Dictating the rating range to be only 3 - 5 so we can see changes in graph

# 3: Number of reviews per month
ax = axes[3]
ax.plot(ts_global["month"], ts_global["num_reviews"], marker="o", linestyle="-")
ax.set_xlabel("Review month")
ax.set_ylabel("# Reviews")
ax.set_title("Number of reviews per month")
ax.tick_params(axis="x", rotation=45)

# 4–6: Average review rating over time (split into 3 time periods)
period_order = ["pre-2016", "2016-2019", "2020+"]
for idx, label in enumerate(period_order, start=4):
    ax = axes[idx]
    ts = time_stats_period[time_stats_period["period"] == label].copy()
    ts = ts[ts["num_reviews"] >= min_reviews]
    if ts.empty:
        ax.set_visible(False)
        continue  # in case the early period has no data

    ts = ts.sort_values("month")
    ax.plot(ts["month"], ts["avg_rating"], marker="o", linestyle="-")
    ax.set_xlabel("Review month")
    ax.set_ylabel("Average rating")
    ax.set_title(f"Average review rating over time ({label})")
    ax.tick_params(axis="x", rotation=45)
    ax.set_ylim(3.0, 5.1)  # focusing on reasonable ratings // can change

# 7: Price vs rating
ax = axes[7]
ax.boxplot(
    [cafe_avg_price[cafe_avg_price["price_num"]==k]["avg_user_rating"] for k in levels],
    labels=[int(k) for k in levels]
)
ax.set_xlabel("Price level (# of $)")
ax.set_ylabel("Average review rating")
ax.set_title("Price vs rating (cafe-level)")

# 8: Mean rating by price level
ax = axes[8]
ax.bar(means.index.astype(int), means.values)
ax.set_xlabel("Price level (# of $)")
ax.set_ylabel("Mean rating")
ax.set_title("Mean rating by price level")

fig.suptitle("Cafe Data Visualizations", fontsize=16, y=0.98)
fig.tight_layout()
plt.show()


In [ ]:
# Folium Visualization:

cafes_map = cafes.dropna(subset=["latitude","longitude"]).copy()
cafes_map["avg_rating"] = pd.to_numeric(cafes_map["avg_rating"], errors="coerce")

# base map
map = folium.Map(
    location=[36.5, -119.5],
    zoom_start=6
)

# california outline from github
ca_geojson_url = "https://raw.githubusercontent.com/glynnbird/usstatesgeojson/master/california.geojson"

folium.GeoJson(
    ca_geojson_url,
    name="California outline",
    style_function=lambda feature: {
        "fillColor": "#ffffff",
        "color": "black",
        "weight": 3,
        "fillOpacity": 0.05
    }
).add_to(map)

# california counties from github
counties_url = "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/california-counties.geojson"
counties_geo = requests.get(counties_url).json()

# converting counties to polygons for averages
county_polys = []
for feat in counties_geo["features"]:
    county_name = feat["properties"]["name"]
    poly = prep(shape(feat["geometry"]))
    county_polys.append((county_name, poly))

# assigning county to each cafe based on lat/long
def find_county(lat, long):
    pt = Point(long, lat)
    for cname, poly in county_polys:
        if poly.contains(pt):
            return cname
    return np.nan

cafes_map["county"] = cafes_map.apply(
    lambda r: find_county(r["latitude"], r["longitude"]),
    axis=1
)

# getting average rating per county
county_stats = (
    cafes_map.dropna(subset=["county"]) # removes cafes without county label so they aren't computed in average
             .groupby("county")
             .agg(avg_rating=("avg_rating", "mean"))
             .reset_index()
)

# choropleth coloring counties by average rating
folium.Choropleth(
    geo_data=counties_geo,
    name="Average rating per county",
    data=county_stats,
    columns=["county", "avg_rating"],
    key_on="feature.properties.name",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.3,
    nan_fill_color="white",
    legend_name="Average cafe rating"
).add_to(map)

# cafe markers
cluster = MarkerCluster(name="Cafe markers").add_to(map)

for _, r in cafes_map.iterrows():
    popup = (
        f"{r.get('name','')}"
        f"<br>Rating: {r.get('avg_rating',np.nan)}"
        f"<br>Price: {r.get('price','')}"
        f"<br>County: {r.get('county','')}"
    )
    folium.CircleMarker(
        location=[r["latitude"], r["longitude"]],
        radius=2.5,
        color="black",
        weight=0.5,
        fill=True,
        fill_opacity=0.7,
        popup=popup
    ).add_to(cluster)

# Toggle panel so we can show/hide features
folium.LayerControl(collapsed=False).add_to(map)
map


In [ ]:
### SECTION 2 - Data Processing ###
# This section processes the data to prepare it for the model

In [ ]:
# Bin Encoding for Hours Throughout Week
def hours_to_onehot(datum):
    feature_hours = [0]*24*7        # Feature vector is 168 wide (24 hours by 7 days)
    hours = ast.literal_eval(datum['hours'])

    weekday_map = {
        "Monday" : 0,
        "Tuesday" : 1,
        "Wednesday" : 2,
        "Thursday" : 3,
        "Friday" : 4,
        "Saturday" : 5,
        "Sunday" :6
    }

    for entry in hours:
        day = weekday_map[entry[0]] # Gets day of week number

        # Closed, skip entry
        if entry[1] == "Closed":
            continue

        # Open 24 hours, all ones
        if entry[1] == "Open 24 hours":
            for d in range(day*24,day*24+24): feature_hours[d] = 1
            continue

        # Converts entry to range of hours
        open_str, close_str = entry[1].split("–")
        start_hr = int(np.floor(parse_time(open_str)))
        end_hr = int(np.ceil(parse_time(close_str)))

        for hr in range(start_hr, end_hr):
            index = day * 24 + hr
            feature_hours[index] = 1

    return feature_hours


# One Hot Encoding for Price
def price_to_onehot(datum):
    feature_price = [0]*3
    if datum['price'] is not np.nan:
        feature_price[len(datum['price'])-1] += 1
    return feature_price

# One Hot Encoding for County
def county_to_onehot(datum):
    # To check functionality, chose entry 863 which is SFO - should map to San Mateo County
    feature_county =[0]*58  # 58 counties in Cali

    index = get_county(datum['latitude'],datum['longitude'])
    feature_county[index] = 1

    return feature_county

# One Hot Encoding for Unix Time Weekday
def unix_weekday_to_onehot(datum):
    feature_weekday = [0]*7

    day = datetime.fromtimestamp(datum['time'] / 1000, tz=timezone.utc).weekday()
    print(day)
    feature_weekday[day] = 1

    return feature_weekday

# One Hot Encoding for Unix Time Hour
def unix_hour_to_onehot(datum):
    feature_dayhour = [0]*24

    hr = datetime.fromtimestamp(datum['time'] / 1000, tz=timezone.utc).hour
    feature_dayhour[hr] = 1

    return feature_dayhour

In [24]:
# Example
example = cafes.iloc[863,:]
print(example)
print("County: ", get_county_name(example['latitude'], example['longitude']))
print("Price encoding: ", price_to_onehot(example))
print("County encoding: ", county_to_onehot(example))
print("Hour encoding: ", hours_to_onehot(example))


gmap_id                       0x808f7790a747d34d:0x368adb21c5e27122
name              Green Beans Coffee - San Francisco Internation...
latitude                                                  37.616889
longitude                                               -122.389291
category                                            ['Coffee shop']
avg_rating                                                      2.1
num_of_reviews                                                    8
price                                                             $
hours             [['Wednesday', 'Open 24 hours'], ['Thursday', ...
Name: 863, dtype: object
County:  40    San Mateo
Name: NAME, dtype: object
Price encoding:  [1, 0, 0]
County encoding:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Hour encoding:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [25]:
example = reviews.iloc[863, :]
print("Unix weekday encoding: ", unix_weekday_to_onehot(example))
print("Unix hour encoding: ", unix_hour_to_onehot(example))

1
Unix weekday encoding:  [0, 1, 0, 0, 0, 0, 0]
Unix hour encoding:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [7]:
# Basic Model - Global Average
global_avg = avg_rating(cafes)
total = 0
for i,row in cafes.iterrows():
    total += (row['avg_rating'] - global_avg)**2

print("Baseline MSE: ", total/len(cafes))

Baseline MSE:  0.16932925119017256


In [ ]:
# Example
example = cafes.iloc[863,:]
print(example)
print("County: ", get_county_name(example['latitude'], example['longitude']))
print("Price encoding: ", price_to_onehot(example))
print("County encoding: ", county_to_onehot(example))
print("Hour encoding: ", hours_to_onehot(example))


In [ ]:
example = reviews.iloc[863, :]
print("Unix weekday encoding: ", unix_weekday_to_onehot(example))
print("Unix hour encoding: ", unix_hour_to_onehot(example))